* pip install datasets transformers evaluate jiwer
* import torch, import datasets, import evaluate, import numpy as np, from dataclasses import data class, field. from typing import Any, Dict, List, Optional, Union
* load train and test splits of mozilla-foundation/common voice 11 0” assamese dataset
* for test, take only first 10 examples and proceed (otherwise you will hit Out of Memory Error)
* from transformers import WhisperFeatureExtractor
* Load feature extractor from the pretrained whisper tiny of openai
* from transformers import WhisperTokenizer
* load tokenizer of openai whisper tiny for assamese for asr task
* from transformers import WhisperProcessor
* load processor of openai whisper tiny for assamese for asr task
* from datasets import Audio
* convert audio to sampling rate of 16k
* write a function which takes batches of input data and gives batches with features    extracted and corresponding labels from the tokenizer.
* from transformers import WhisperForConditionalGeneration
* model = WhisperForConditionalGeneration.from pretrained(”openai/whisper-tiny”)
* set language to assamese, task to asr and forced decoder ids to none
* Write a data collator
* import evaluate and set metric = evaluate.load(”wer”)
* write a function def compute metrics(pred) which takes predictions and returns wer
* from transformers import Seq2SeqTrainingArguments
* set the arguments for Seq2SeqTrainingArguments() as mentioned in the link `https://huggingface.co/blog/fine-tune-whisper` but change warmup steps to 50, max steps to 100, per device train batch size=8, per device eval batch size=1, save steps=100, eval steps=100
* from transformers import Seq2SeqTrainer
* Set the arguments based on the info mentioned in the above link
* start training


In [1]:
import torch
import datasets
import evaluate
import numpy as np
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

In [2]:
# load train and test splits of mozilla-foundation/common-voice-11-0 Assamese
ds_train = datasets.load_dataset('mozilla-foundation/common_voice_11_0', 'as', split='train')
ds_test = datasets.load_dataset('mozilla-foundation/common_voice_11_0', 'as', split='train[:10]')

README.md:   0%|          | 0.00/14.4k [00:00<?, ?B/s]

common_voice_11_0.py:   0%|          | 0.00/8.13k [00:00<?, ?B/s]

languages.py:   0%|          | 0.00/3.44k [00:00<?, ?B/s]

release_stats.py:   0%|          | 0.00/60.9k [00:00<?, ?B/s]

The repository for mozilla-foundation/common_voice_11_0 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/mozilla-foundation/common_voice_11_0.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


...


Reading metadata...: 308it [00:00, 237253.56it/s]


Generating other split: 0 examples [00:00, ? examples/s]


Reading metadata...: 297it [00:00, 268819.22it/s]


Generating invalidated split: 0 examples [00:00, ? examples/s]


Reading metadata...: 163it [00:00, 257309.58it/s]


In [3]:
ds_train

Dataset({
    features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
    num_rows: 824
})

In [4]:
ds_test

Dataset({
    features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
    num_rows: 10
})

In [5]:
# from transformers import WhisperFeatureExtractor
# Load feature extractor from the pretrained whisper tiny of openai

from transformers import WhisperFeatureExtractor
whisper_feat_extractor = WhisperFeatureExtractor.from_pretrained('openai/whisper-tiny')

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

In [6]:
# from transformers import WhisperTokenizer
# load tokenizer of openai whisper tiny for assamese for asr task
from transformers import WhisperTokenizer
whisper_tokenizer = WhisperTokenizer.from_pretrained('openai/whisper-tiny', language='assamese', task='transcribe')

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

In [8]:
# load processor of openai whisper tiny for assamese for asr task
from transformers import WhisperProcessor
whisper_processor = WhisperProcessor.from_pretrained('openai/whisper-tiny', language='assamese', task='transcribe')

---
1. How many examples are present in the train split of mozilla-foundation/common voice 11 0 ”assamese”
langauge dataset ?

In [15]:
ds_train

Dataset({
    features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
    num_rows: 824
})

2. How many unique characters are there in the train split text ?

In [17]:
chars = set()
for sample in ds_train:
    chars.update(sample['sentence'])

In [18]:
len(chars)

74

In [19]:
chars

{' ',
 '!',
 '"',
 "'",
 ',',
 '-',
 '.',
 '?',
 '।',
 'ঁ',
 'ং',
 'ঃ',
 'অ',
 'আ',
 'ই',
 'ঈ',
 'উ',
 'এ',
 'ও',
 'ঔ',
 'ক',
 'খ',
 'গ',
 'ঘ',
 'ঙ',
 'চ',
 'ছ',
 'জ',
 'ঞ',
 'ট',
 'ঠ',
 'ড',
 'ঢ',
 'ণ',
 'ত',
 'থ',
 'দ',
 'ধ',
 'ন',
 'প',
 'ফ',
 'ব',
 'ভ',
 'ম',
 'য',
 'র',
 'ল',
 'শ',
 'ষ',
 'স',
 'হ',
 '়',
 'া',
 'ি',
 'ী',
 'ু',
 'ূ',
 'ৃ',
 'ে',
 'ৈ',
 'ো',
 'ৌ',
 '্',
 'ৎ',
 'ড়',
 'ঢ়',
 'য়',
 'ৰ',
 'ৱ',
 '৷',
 '‘',
 '’',
 '“',
 '”'}

3. What is the sampling rate of the original mozilla-foundation/common voice 11 0 ”assamese” language audio in Hz?


In [22]:
sr = ds_train.map(lambda x: {'sampling_rate': x['audio']['sampling_rate']}, remove_columns=ds_train.column_names)
set(sr['sampling_rate'])

Map:   0%|          | 0/824 [00:00<?, ? examples/s]

{48000}

In [23]:
# Let's try for test, too
sr = ds_test.map(lambda x: {'sampling_rate': x['audio']['sampling_rate']}, remove_columns=ds_train.column_names)
set(sr['sampling_rate'])

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

{48000}

4. What is the format of the mozilla-foundation/common voice 11 0 ”assamese” language audio ?

In [26]:
import os.path as op

audio_format = ds_train.map(lambda x: {'ext': op.splitext(x['audio']['path'])[-1]}, remove_columns=ds_train.column_names)
set(audio_format['ext'])

Map:   0%|          | 0/824 [00:00<?, ? examples/s]

{'.mp3'}

In [27]:
# Let's try on test also
audio_format = ds_test.map(lambda x: {'ext': op.splitext(x['audio']['path'])[-1]}, remove_columns=ds_train.column_names)
set(audio_format['ext'])

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

{'.mp3'}

5. What will be the window length in msec if n fft is 400 in ”WhisperFeatureExtractor” ?

In [50]:
# The window length in seconds is 400 / sampling_rate, in msec it is *= 1000
400 / whisper_feat_extractor.sampling_rate * 1000

25.0

In [30]:
#  What is the first token number after tokenising the 56th example?
enc = whisper_tokenizer(ds_train[56]['sentence'])
enc[:3]

{'input_ids': [50258, 50350, 50359], 'attention_mask': [1, 1, 1]}

7. What is the token corresponding to the token number 51833 in whisper?

In [32]:
id2token = {v: k for k, v in whisper_tokenizer.get_vocab().items()}
id2token[51833]

'<|29.38|>'

8. Is token number 51833 a special token?

In [35]:
vocab = whisper_tokenizer.get_vocab()

In [36]:
vocab['<|29.38|>']

51833

In [43]:
51833 in whisper_tokenizer.all_special_ids

False

9. What is the token corresponding to the token number 50350 in whisper?

In [46]:
id2token[50350]

'<|as|>'

10. Is token number 50350 a special token?

In [47]:
50350 in whisper_tokenizer.all_special_ids

True

In [49]:
whisper_feat_extractor.sampling_rate

16000